#Project Guidelines

1. Choose the data set of your interest.  Make sure your data set has required features such as contents for content-based filtering and user-item interactions (ratings or similar measure) for collaborating filtering.

2. Data cleaning and preprocessing is a crucial component of the project. Reflect sufficient efforts on this section on your project.

3. Build three different recommender systems:

**Content-based recommender system**

At the end of the work, you will define a function such as content_based_recommender( ) which takes two inputs: title/item name  and number of recommendations and the output: a list of recommended title/item name similar to the input title/item.

**Collaborative based recommender system using either knn or svd**

At  the end of the work, you will define a function such as collaborative_recommender( ) which takes two inputs: user_id & number of recommendations and the output: a list of recommended title/item recommended for the user.

**Hybrid recommender system**

You will combine both content-based and collaborative filtering ( as we did in class). You will define a function such as hybrid_recommender( ) which takes three inputs: title/item, user_id, and number of recommended item based on given item and user

## Grading Criteria

1. Project description (3 points): Explain your dataset, features that your are including in your model, and goal of your project.

2. Organization of the notebook file (3 points):  Have an appropriate title for the project,  headlines and their levels for each section so that you can squeeze into one chunk for each model. Write a clear comment for each chunk of code explaining the operations/functions of them.

3. Data cleaning and preprocessing ( 7 points) :

4. Efforts and practicality of the model ( 7 points):  Make sure your project reflects the effort you made to get the best possible result. Also, make sure the efficiency of your model. I recommend to go over the reference text book "Hands-On Recommendation Systems with Python" which is available online in our MK library.



#Creating Online Retail Recommender Systems
- Dataset source: https://archive.ics.uci.edu/dataset/352/online+retail

##Project Description

**Explain your dataset:** This data set is transnational and contains the transactions from a UK-based online retail company occuring between 1/12/2010 and 9/12/2011. This business sells "unique all-occasion gifts".

**Features in this dataset:**

*   InvoiceNo - 6-digit unique number assigned to each transaction. Starting with a 'c' indicates a cancellation
*   StockCode - 5-digit unique number assigned to each product
*   Description - Name of product
*   Quantity - Quantity of item purchased per transaction
*   InvoiceDate - Date and time of transaction
*   UnitPrice - Product price of a single unit
*   CustomerID - 5-digit unique number assigned to each customer
*   Country - customers country of residence

**Features used in the model**

InvoiceNo, Description, StockCode , UnitPrice, CustomerID, Country, Quantity, and month. The month of purchase was extracted from the InvoiceDate to give a broader generalization of seasonal buying.



**Goal of this project:** The goal of this project is to create a content-based, collabrative, and hybrid recommender system that recommends diverse products to a user in order to encourage more pruchases.

##Reading data:

In [ ]:
#mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# read data for Julia
#import pandas as pd
#retail= pd.read_csv('/content/drive/MyDrive/DSC201/Project1/OnlineRetail.csv')

In [ ]:
# read data for Josie
import pandas as pd
retail= pd.read_csv('/content/drive/MyDrive/OnlineRetail.csv')

In [ ]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [ ]:
# there is 541909 items and 8 descriptors
retail.shape

(541909, 8)

In [ ]:
# 8 descriptors
retail.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

##Data Cleaning and Preproccessing:

In [ ]:
retail.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [ ]:
#checking for missing values
retail.isnull()
number_of_missing_values = retail.isnull().sum()
number_of_missing_values

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

Handling missing data:

In [ ]:
#drop all rows that are missing a costumer ID
df = retail[retail['CustomerID'].notna()]
df.shape

(406829, 8)

In [ ]:
#checking for missing values
df.isnull()
number_of_missing_values = df.isnull().sum()
number_of_missing_values

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Checking for invalid values:

In [ ]:
unique_values = sorted(df['Quantity'].unique())
print(unique_values) #cannot have negative quanity

[-80995, -74215, -9360, -3114, -2000, -1930, -1515, -1350, -1300, -1296, -1200, -960, -840, -828, -756, -720, -701, -670, -624, -600, -576, -500, -480, -468, -432, -420, -408, -400, -360, -334, -324, -318, -312, -300, -288, -276, -252, -250, -244, -240, -234, -220, -200, -192, -186, -184, -180, -168, -164, -162, -160, -152, -150, -144, -140, -132, -130, -126, -121, -120, -113, -112, -110, -108, -102, -100, -96, -94, -86, -85, -82, -80, -79, -76, -75, -72, -70, -68, -67, -66, -64, -60, -58, -53, -52, -51, -50, -49, -48, -47, -46, -45, -43, -42, -40, -39, -38, -37, -36, -35, -34, -33, -32, -31, -30, -29, -28, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 65, 66, 67, 69, 

In [ ]:
value_counts = df['Quantity'].value_counts()
value_counts

 1        73314
 12       60033
 2        58003
 6        37688
 4        32183
          ...  
 828          1
 560          1
-408          1
 512          1
-80995        1
Name: Quantity, Length: 436, dtype: int64

In [ ]:
df = df[df['Quantity'] >= 0]
df.shape

(397924, 8)

lost 8905 values

In [ ]:
country_counts = df['Country'].value_counts()
country_counts

United Kingdom          354345
Germany                   9042
France                    8342
EIRE                      7238
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1842
Portugal                  1462
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     248
Unspecified                244
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon                     45
Lithuani

In [ ]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [ ]:
# Extract month from invoicedate to generalize what month people bought certain things
# - then drop invoiceDate
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%y %H:%M')
#chatgpt used to figure out how to extract month

# Extract month, make new column, and convert to float
df['month'] = df['InvoiceDate'].dt.month.astype(float)

In [ ]:
df = df.drop(columns=['InvoiceDate'])

In [ ]:
# Convert InvoiceNo to float
df['InvoiceNo'] = df['InvoiceNo'].astype(float)

In [ ]:
df.dtypes

InvoiceNo      float64
StockCode       object
Description     object
Quantity         int64
UnitPrice      float64
CustomerID     float64
Country         object
month          float64
dtype: object

**General information about clean data**

In [ ]:
df.CustomerID.nunique() #number of unique customers

4339

In [ ]:
df.Description.nunique()

3877

In [ ]:
df.StockCode.nunique() #number of unique products

3665

proof stockcode and description are not entirely redundant features so keep both

In [ ]:
df.Quantity.nunique() #number of unique Quantities

302

In [ ]:
int(df.Quantity.mean()) #average number of products bought per customer trip

13

In [ ]:
max =df.Quantity.max()
min= df.Quantity.min()

print(min,"-",max, "items")

1 - 80995 items


In [ ]:
int(df.UnitPrice.mean()) #average price was $3

3

In [ ]:
max =df.UnitPrice.max()
min= df.UnitPrice.min()

print("$",min,"-",max, )

$ 0.0 - 8142.75


In [ ]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,month
0,536365.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,12.0
1,536365.0,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,12.0
2,536365.0,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,12.0
3,536365.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,12.0
4,536365.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,12.0


In [ ]:
# make clean data a csv + download it
from google.colab import files
file_name = "df.csv"

# save the file to Colab's file system
df.to_csv(file_name, index=False)

#download the file to local machine
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Recommender Systems:

### **Clean data as new DF**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# read df for julia
#import pandas as pd
#df= pd.read_csv('/content/drive/MyDrive/DSC201/Project1/df.csv')

In [ ]:
# read df for josie
import pandas as pd
df= pd.read_csv('/content/drive/MyDrive/df.csv')

In [ ]:
# check data frame
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,month
0,536365.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,17850.0,United Kingdom,12.0
1,536365.0,71053,WHITE METAL LANTERN,6,3.39,17850.0,United Kingdom,12.0
2,536365.0,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,17850.0,United Kingdom,12.0
3,536365.0,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,17850.0,United Kingdom,12.0
4,536365.0,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,17850.0,United Kingdom,12.0


## **Content Based Recommender System**

In [ ]:
# determine what features we want to base recommendations off of
features = ['InvoiceNo','Description', 'StockCode', 'UnitPrice', 'CustomerID', 'Country','Quantity','month']


In [ ]:
def combined_features(row):
   return str(row['InvoiceNo'])+ " "+str(row['StockCode'])+" " +str(row['UnitPrice'])+" "+str(row['CustomerID'])+"  "+str(row['Country'])+"  "+str(row['Description'])+"  "+str(row['Quantity'])+"  "+str(row['month'])

In [ ]:
df["combined_features"] = df.apply(combined_features, axis =1) #create new column combined features

In [ ]:
df['combined_features'].head(10)

0    536365.0 85123A 2.55 17850.0  United Kingdom  ...
1    536365.0 71053 3.39 17850.0  United Kingdom  W...
2    536365.0 84406B 2.75 17850.0  United Kingdom  ...
3    536365.0 84029G 3.39 17850.0  United Kingdom  ...
4    536365.0 84029E 3.39 17850.0  United Kingdom  ...
5    536365.0 22752 7.65 17850.0  United Kingdom  S...
6    536365.0 21730 4.25 17850.0  United Kingdom  G...
7    536366.0 22633 1.85 17850.0  United Kingdom  H...
8    536366.0 22632 1.85 17850.0  United Kingdom  H...
9    536367.0 84879 1.69 13047.0  United Kingdom  A...
Name: combined_features, dtype: object

In [ ]:
# change text in combined features to vector form
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(df['combined_features'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#movie titles to index in the dataframe for quick look-up
item_index = pd.Series(df.index, index=df['Description']).to_dict()

In [ ]:
def get_index_from_description(description):
    return item_index[description]

def get_description_from_index(index):
    return df['Description'].iloc[index]

In [ ]:
def content_based_recommender(item_description, num_of_rec=10):
    item_idx = get_index_from_description(item_description)

    # item description to vector
    description_vec = vectorizer.transform([df['combined_features'].iloc[item_idx]])

    # cosine similarity of picked item with other items
    cosine_sim = cosine_similarity(description_vec, count_matrix)

    # getting recommendations
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    recommended_indices = []
    recommended_descriptions = set()
    for idx, score in sim_scores:
        if df['Description'].iloc[idx] not in recommended_descriptions:
            recommended_indices.append(idx)
            recommended_descriptions.add(df['Description'].iloc[idx])
            if len(recommended_indices) == num_of_rec + 1:
                break

    recommendations = [get_description_from_index(idx) for idx in recommended_indices[1:]]
    return recommendations

In [ ]:
# final output
recommended_items = content_based_recommender('WHITE METAL LANTERN', 5)

print(f" 5 similar items for you based on your interest in WHITE METAL LANTERN:")
print("**********************************************************")
for idx, item in enumerate(recommended_items):
    print(f"{idx+1}. {item}")
print("************************************************************")

 5 similar items for you based on your interest in WHITE METAL LANTERN:
**********************************************************
1. WHITE MOROCCAN METAL LANTERN
2. BISCUIT TIN VINTAGE GREEN
3. BISCUIT TIN VINTAGE RED
4. PAINTED METAL PEARS ASSORTED
5. REGENCY CAKESTAND 3 TIER
************************************************************


## **Collaborative Filtering Based Recommender System**

User-item matrix breakdown:
*   Rows = unique CustomerIDs (users).
*   Columns = unique Descriptions (items).
*   Values = Quantity of each item purchased by a user

In [ ]:
#user-item interaction matrix
user_item_matrix = df.pivot_table(index='CustomerID',
                                  columns='Description',
                                  values='Quantity',
                                  aggfunc='sum',
                                  fill_value=0)

#  basic info + few rows of matrix
(user_item_matrix.info(), user_item_matrix.head())

<class 'pandas.core.frame.DataFrame'>
Float64Index: 4339 entries, 12346.0 to 18287.0
Columns: 3877 entries,  4 PURPLE FLOCK DINNER CANDLES to ZINC WIRE SWEETHEART LETTER TRAY
dtypes: int64(3877)
memory usage: 128.4 MB


(None,
 Description   4 PURPLE FLOCK DINNER CANDLES   50'S CHRISTMAS GIFT BAG LARGE  \
 CustomerID                                                                    
 12346.0                                   0                               0   
 12347.0                                   0                               0   
 12348.0                                   0                               0   
 12349.0                                   0                               0   
 12350.0                                   0                               0   
 
 Description   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK  \
 CustomerID                                                      
 12346.0                       0                             0   
 12347.0                       0                             0   
 12348.0                       0                             0   
 12349.0                       0                             0   
 12350.0                       0   

* Note: Matrix is sparse, meaning there are many zeros. This is because most customers are only buying one or a few items.

### Exploring KNN and SVD

**K Nearest Neighbor**

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [ ]:
# Initialize the KNN model
knn_model = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')

# fit the model to the user-item matrix
knn_model.fit(user_item_matrix.values)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [ ]:
# Function to get recommendations
def knn_collab_rec(customer_id, user_item_matrix, knn_model, k=5):
    # index of the user
    user_index = np.where(user_item_matrix.index == customer_id)[0][0]

    #  k-nearest neighbors
    distances, indices = knn_model.kneighbors(user_item_matrix.iloc[user_index, :].values.reshape(1, -1),
                                              n_neighbors=k+1)

    # customer IDs of neighbors
    neighbor_ids = user_item_matrix.iloc[indices.flatten(), :].index

    # remove user's own ID from neighbors
    neighbor_ids = neighbor_ids[neighbor_ids != customer_id]

    # id items that the neighbors have interacted with
    neighbor_items = user_item_matrix.loc[neighbor_ids, :]

    # sum the quantities purchased by neighbors and recommend items with highest total quantity
    recommended_items = neighbor_items.sum(axis=0).sort_values(ascending=False)

    # exclude items that the user has already interacted with!!
    user_interacted_items = user_item_matrix.loc[customer_id, :][user_item_matrix.loc[customer_id, :] > 0].index
    recommended_items = recommended_items[~recommended_items.index.isin(user_interacted_items)]

    # recommend top k items
    recommended_items = recommended_items.head(k).index.tolist()

    return recommended_items


In [ ]:
# test function w/ a user ID
knn_collab_rec(12347, user_item_matrix, knn_model, k=5)

['DOUGHNUT LIP GLOSS ',
 'GROW A FLYTRAP OR SUNFLOWER IN TIN',
 '10 COLOUR SPACEBOY PEN',
 'PACK OF 12 SPACEBOY TISSUES',
 'SET/10 PINK POLKADOT PARTY CANDLES']

**Singular Value Decomposition**

In [ ]:
# imports
from scipy.sparse.linalg import svds

In [ ]:
# make data type float
user_item_matrix_float = user_item_matrix.astype('float')

In [ ]:
# Singular Value Decomposition
U, sigma, Vt = svds(user_item_matrix_float.values, k=50)  # k is the number of latent factors

In [ ]:
# sigma to a diagonal matrix
sigma_diag_matrix = np.diag(sigma)

In [ ]:
# predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma_diag_matrix), Vt)

#  shape of the predicted ratings (to compare to og make sure have all features)#hooray
predicted_ratings.shape, user_item_matrix.shape

((4339, 3877), (4339, 3877))

In [ ]:
def svd_collab_rec(user_id, num_recommendations=5):
    # check if user_id is in the user-item matrix
    if user_id not in user_item_matrix.index:
        raise ValueError("User ID not found in the user-item matrix.")

    # getting the index of user_id in the user-item matrix
    user_idx = user_item_matrix.index.get_loc(user_id)

    # predicted ratings for the user
    user_predicted_ratings = predicted_ratings[user_idx, :]

    # Id items that the user has not interacted with
    user_actual_ratings = user_item_matrix.loc[user_id, :]
    unrated_items_mask = user_actual_ratings == 0
    unrated_items_idx = np.where(unrated_items_mask)[0]

    # predicted ratings for unrated items and id top-rated items
    unrated_predicted_ratings = user_predicted_ratings[unrated_items_idx]
    top_rated_items_idx = unrated_items_idx[np.argsort(-unrated_predicted_ratings)[:num_recommendations]]

    # descriptions of top-rated items
    recommended_items = user_item_matrix.columns[top_rated_items_idx].tolist()

    return recommended_items


In [ ]:
svd_collab_rec(12347, num_recommendations=5)

['PACK OF 72 RETROSPOT CAKE CASES',
 'PACK OF 72 SKULL CAKE CASES',
 'PACK OF 12 TRADITIONAL CRAYONS',
 'Manual',
 'DOLLY GIRL LUNCH BOX']

**Comparing Collaborative Models**




In [ ]:
# items bought by customer 12356
customer_purchases = df[df['CustomerID'] == 12356]

# count frequency of each item purchased by Customer
item_counts = customer_purchases['Description'].value_counts()

# 5 most frequently purchased items for customer
top_5_items = item_counts.head(5)

# results
print("Top 5 most popular items for selected Customer:")
for i, (item, count) in enumerate(top_5_items.items()):
    print(f"{i+1}. {item} (Purchased {count} times)")

Top 5 most popular items for selected Customer:
1. RED RETROSPOT CAKE STAND (Purchased 3 times)
2. REGENCY CAKESTAND 3 TIER (Purchased 3 times)
3. SET OF 3 HEART COOKIE CUTTERS (Purchased 2 times)
4. 60 TEATIME FAIRY CAKE CASES (Purchased 2 times)
5. ASSORTED EASTER GIFT TAGS (Purchased 1 times)


In [ ]:
# test function w/ a user ID
knn_collab_rec(12356, user_item_matrix, knn_model, k=5)

['PACK OF 60 DINOSAUR CAKE CASES',
 'PACK OF 60 PINK PAISLEY CAKE CASES',
 'WORLD WAR 2 GLIDERS ASSTD DESIGNS',
 'MULTICOLOUR SPRING FLOWER MUG',
 'POLYESTER FILLER PAD 40x40cm']

In [ ]:
svd_collab_rec(12356, num_recommendations=5)

['PACK OF 60 DINOSAUR CAKE CASES',
 'PACK OF 60 PINK PAISLEY CAKE CASES',
 "YOU'RE CONFUSING ME METAL SIGN ",
 'HOT BATHS METAL SIGN',
 'CHRISTMAS PUDDING TRINKET POT ']

**Conclusion:**
As both provide similar results, we have chosen to use SVD because it requires less computational power and is better with sparse data, which applies to our data set.

### **Final Collaborative Model made with SVD**

In [ ]:
def collaborative_recommender(user_id, num_recommendations=5):
    # check if user_id is in the user-item matrix
    if user_id not in user_item_matrix.index:
        raise ValueError("User ID not found in the user-item matrix.")

    # getting the index of user_id in the user-item matrix
    user_idx = user_item_matrix.index.get_loc(user_id)

    # predicted ratings for the user
    user_predicted_ratings = predicted_ratings[user_idx, :]

    # Id items that user has NOT interacted with
    user_actual_ratings = user_item_matrix.loc[user_id, :]
    unrated_items_mask = user_actual_ratings == 0
    unrated_items_idx = np.where(unrated_items_mask)[0]

    # predicted ratings for the unrated items & id top-rated items
    unrated_predicted_ratings = user_predicted_ratings[unrated_items_idx]
    top_rated_items_idx = unrated_items_idx[np.argsort(-unrated_predicted_ratings)[:num_recommendations]]

    # descriptions of the top-rated items
    recommended_items = user_item_matrix.columns[top_rated_items_idx].tolist()

    # returns recs
    return recommended_items

In [ ]:
recommended_items = collaborative_recommender(12356, 5)

print(f"Recommended items for user 12356:")
print("**********************************************************")
for i, item_description in enumerate(recommended_items, 1):
    print(f"{i}. {item_description}")
print("************************************************************")

Recommended items for user 12356:
**********************************************************
1. PACK OF 60 DINOSAUR CAKE CASES
2. PACK OF 60 PINK PAISLEY CAKE CASES
3. YOU'RE CONFUSING ME METAL SIGN 
4. HOT BATHS METAL SIGN
5. CHRISTMAS PUDDING TRINKET POT 
************************************************************


## **Hybrid Recommender System**

In [ ]:
def hybrid_recommendation(user_id, item_description, num_recommendations=5):
    # Get recommendations from content-based model
    content_recommendations = content_based_recommender(item_description, num_recommendations*2)

    # Get recommendations from collaborative model
    collaborative_recommendations = collaborative_recommender(user_id, num_recommendations*2)

    # combine recs and make sure no duplicates
    combined_recommendations = list(set(content_recommendations + collaborative_recommendations))

    # if too many recs - choose the top N
    if len(combined_recommendations) > num_recommendations:
        combined_recommendations = combined_recommendations[:num_recommendations]

    # prints recs
    print(f"\nRecommended items for user {user_id}, based on item {item_description}:")
    print("**********************************************************")
    for i, reco_item_description in enumerate(combined_recommendations, 1):
        print(f"{i}. {reco_item_description}")
    print("************************************************************")

In [ ]:
user_id = 12356  # Example userID
item_description = 'WHITE METAL LANTERN'  # Example description
num_recommendations = 5  # Number of recs

#Hybrid recs
hybrid_recommendation(user_id, item_description, num_recommendations)


Recommended items for user 12356, based on item WHITE METAL LANTERN:
**********************************************************
1. REGENCY CAKESTAND 3 TIER
2. YOU'RE CONFUSING ME METAL SIGN 
3. WHITE MOROCCAN METAL LANTERN
4. WHITE HANGING HEART T-LIGHT HOLDER
5. HANGING METAL STAR LANTERN
************************************************************


### Nonfunctioning Hybrid Recommender System - requires too much RAM !!
- An example of inefficiency

In [ ]:
df.dtypes

InvoiceNo            float64
StockCode             object
Description           object
Quantity               int64
UnitPrice            float64
CustomerID           float64
Country               object
month                float64
combined_features     object
dtype: object

In [ ]:
# choosing content to recommend, price must be converted to same dtype as description
df['content']= df['UnitPrice'].astype(str)+ " "+df['Description']

In [ ]:
df['content']

0          2.55 WHITE HANGING HEART T-LIGHT HOLDER
1                         3.39 WHITE METAL LANTERN
2              2.75 CREAM CUPID HEARTS COAT HANGER
3         3.39 KNITTED UNION FLAG HOT WATER BOTTLE
4              3.39 RED WOOLLY HOTTIE WHITE HEART.
                            ...                   
397919            0.85 PACK OF 20 SPACEBOY NAPKINS
397920            2.1 CHILDREN'S APRON DOLLY GIRL 
397921          4.15 CHILDRENS CUTLERY DOLLY GIRL 
397922        4.15 CHILDRENS CUTLERY CIRCUS PARADE
397923          4.95 BAKING SET 9 PIECE RETROSPOT 
Name: content, Length: 397924, dtype: object

In [ ]:
# Vectorize the content
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
content_matrix = tfidf.fit_transform(df['content'])

In [ ]:
content_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
tfidf.get_feature_names_out().shape

(2115,)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#cosine_sim = cosine_similarity(content_matrix)
#cosine_sim

In [ ]:
def get_index_from_description(Description):
    return df[df.Description == Description].index[0]

In [ ]:
# description of item bought
item_description='DOORMAT NEW ENGLAND'

In [ ]:
def get_description_from_index(index):
    return df[df.index == index]["Description"].values[0]

In [ ]:
get_description_from_index(1)

In [ ]:
pairwise_cosine= list(enumerate(cosine_sim[movie_index]))

In [ ]:
# Selecting first 25 movies with highest cosine value
sorted_cosine = sorted(pairwise_cosine,
                               key=lambda x:x[1],
                               reverse=True)[0:26]

In [ ]:
# Collab part
#user-item interaction matrix
user_item_matrix = df.pivot_table(index='CustomerID', columns='Description', values='Quantity', aggfunc='sum', fill_value=0)

In [ ]:
#  Calculate the mean rating of each user
user_item_mean = user_item_matrix.mean(axis=1)

In [ ]:
# Normalize the user-item rating matrix by subtracting the mean rating of each user
user_item_normalized = user_item_matrix.sub(user_item_mean, axis=0)

In [ ]:
# Convert the DataFrame to a NumPy array
user_item_array = user_item_normalized.values

In [ ]:
# Perform Singular Value Decomposition (SVD)
U, sigma, Vt = svds(user_item_array, k=75)

In [ ]:
# Convert sigma to a diagonal matrix
sigma_diag_matrix = np.diag(sigma)

In [ ]:
# Reconstruct the predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma_diag_matrix), Vt) + user_item_mean.values.reshape(-1, 1)

In [ ]:
# Create a DataFrame for predicted ratings
predicted_ratings_df = pd.DataFrame(predicted_ratings,
                                    columns=user_item_matrix.columns,
                                    index=user_item_matrix.index)

In [ ]:
predicted_ratings_df.head()

In [ ]:
# set customer ID
customer_id=12346.0

In [ ]:
# Get the user's predicted ratings for the given index
user_predicted_ratings = predicted_ratings_df.loc[customer_id]

In [ ]:
unrated_items = user_item_matrix.loc[customer_id][user_item_matrix.loc[customer_id] == 0].index

In [ ]:
predicted_top_rated_items = user_predicted_ratings[unrated_items].sort_values(ascending=False).index

In [ ]:
# Define a function to get movie recommendations for a given user
def get_item_recommendations_svd(customer_id, num_recommendations=5):

    # Get the user's predicted ratings
    user_predicted_ratings = predicted_ratings_df.loc[customer_id]

    # Find movies that the user has not already rated
    unrated_items = user_item_matrix.loc[customer_id][user_item_matrix.loc[customer_id] == 0].index

    # Get the predicted ratings for those unrated movies and sort them in descending order
    top_rated_items = user_predicted_ratings[unrated_items].sort_values(ascending=False).index

    # Choose the top k movies for recommendation
    recommended_items = top_rated_items[:num_recommendations]

    print(f"Recommended items for user {customer_id}:")
    for customerid in recommended_items:
        item_description = df[df['CustomerID'] == customerid]['Description'].values[0]
        print(f"CustomerID: {customerid}, Item: {description}")

    return print(f"\n Enjoy shopping!!!")

In [ ]:
get_item_recommendations_svd(customer_id, num_recommendations=5)

In [ ]:
# hybrid
def hybrid_recommendation(CustomerID, Description, k=10):

  # k: number of recommended items
  cosine_sim = cosine_similarity(content_matrix)

  # get index from the title
  item_index=get_index_from_description(Description)

  # pairwise cosine value for given item index
  pairwise_cosine= list(enumerate(cosine_sim[item_index]))

  # Selecting first 25 items with highest cosine value
  sorted_cosine = sorted(pairwise_cosine,
                               key=lambda x:x[1],
                               reverse=True)[1:26]
  # Store indices for item with high cosine value
  item_indices= [i[0] for i in sorted_cosine]

  # get item id from movie_indices
  item_ids=[get_description_from_index(i) for i in item_indices]

  # Get predicted value of the item for the user
  predicted_rating= [predicted_ratings_df.loc[CustomerID][i] for i in item_ids]

  # making a dataframe with item_ids with predicted value
  item_rating=pd.DataFrame({'item_id': item_ids, 'pred_ratings': predicted_rating})

  # Sort the DataFrame based on 'pred_ratings'
  sorted_item_rating = item_rating.sort_values(by='pred_ratings',ascending=False)

  # Getting list of top k items item id
  top_k_item_ids= sorted_item_rating['item_id'].values[0:k].tolist()

  print(f"Recommended items for user {CustomerID} and item: {Description} are")
  for item_id in top_k_item_ids:
        item_description = df[df['item_id'] == item_id]['Description'].values[0]
        print(f"Item ID: {item_id}, Title: {item_description}")

  return print(f"\n yay items to buy")

In [ ]:
hybrid_recommendation(10, item_description, k=10)

# **Efficiency of our system**

When creating our content-based recommender system, we explored using K-Nearest Neighbors and Singular Value Decomposition to find the best possible recommendations for our dataset. We determined that SVD gave us the best result, and it was more efficient due to it's ability to handle sparse data sets. For our hybrid recommender system, we tried two different approaches to creating the system. The first approach, attempted to create the system differently than the content-based system we made at the beginning. This hybrid system utilized cosine similarity outside of the functions that actually created the model. This forced Google Colab to use too much RAM power, and effectively crashed the program every time. To assess this issue, we used the collaborative and content-based models we initially made because they find cosine similarity inside of methods. This uses less RAM and is therefore more effiecient when working with a dataset as large as we are. In the end, by using SVD and moving where we calculated cosine similarity we were able to create a more efficient and accurate hybrid recommender system